In [15]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [17]:
pinecone_ = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index_name = 'rag'
namespace = ''
pinecone_.create_index(
    name=index_name,
    dimension=1, 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'a272cc4c1ea0fe9e192271951ac374e8', 'Date': 'Thu, 29 Aug 2024 07:46:19 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [20]:
import json
data = json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. Emily Thorpe',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Thorpe's lectures are engaging and informative. She really knows how to make complex topics accessible."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': "Brilliant instructor! Prof. Chen's passion for AI is contagious. His projects are challenging but rewarding."},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Psychology',
  'stars': 3,
  'review': "Dr. Johnson's research is cutting-edge, but her lectures can be a bit dry. Office hours are helpful though."},
 {'professor': 'Prof. David Martinez',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Martinez brings history to life with his storytelling. His exams are tough but fair.'},
 {'professor': 'Dr. Rachel Wong',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Dr. Wong is an excellent teacher. Her lab demonstrations are always exciting and memorable.'},
 {'professor': 'Prof. Jonat

In [21]:
process_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-3-small',
    )
    embedding = response.data[0].embedding
    process_data.append({
        'values': embedding,
        'id': review['professor'],
        'metadata': {
            'review': review['review'],
            'subject': review['subject'],
            'stars': review['stars']
        }
    })

In [22]:
process_data[0]

{'values': [0.0058555184,
  -0.033286087,
  -0.019502947,
  0.013597778,
  0.034927886,
  0.01872177,
  -0.04112434,
  0.03982679,
  0.005875379,
  -0.022680616,
  0.02421649,
  0.012432632,
  0.025699403,
  -0.037814267,
  0.01415387,
  0.020932898,
  -0.008175879,
  -0.01917194,
  0.049862932,
  0.05433815,
  0.04695007,
  -0.0027986662,
  0.031750217,
  -0.020919658,
  -0.059899073,
  -0.021978881,
  0.026996952,
  0.028969754,
  -0.00044603227,
  0.021316867,
  0.05746286,
  -0.013160848,
  -0.018112717,
  -0.028387181,
  0.0052232943,
  0.042580772,
  0.0082950415,
  0.030849876,
  0.025937729,
  0.010651813,
  0.05857504,
  -0.024838785,
  -0.04565252,
  0.025911247,
  0.007970654,
  0.0057893167,
  -0.010790836,
  -0.0054682395,
  0.022574695,
  0.017199136,
  -0.042819098,
  -0.0032157353,
  -0.01780819,
  -0.01878797,
  -0.08023616,
  0.030002497,
  -0.0019893537,
  0.046393976,
  -0.017821431,
  -0.028413663,
  0.020019319,
  0.0005618848,
  -0.0033067623,
  0.012624617,
  -0

In [23]:
index = pinecone_.Index('rag')
index.upsert(
    vectors=process_data,
    namespace="namespace"
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 29 Aug 2024 07:49:26 GMT', 'Content-Type': 'application/json', 'Content-Length': '101', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '996', 'x-pinecone-request-id': '959806087244108349', 'x-envoy-upstream-service-time': '37', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 1536 does not match the dimension of the index 1","details":[]}


In [24]:
index.describe_index_stats()

{'dimension': 1,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}